In [1]:
import numpy as np
import six,nestle
from sncosmo.utils import Result, Interp1D,ppf
from scipy.interpolate import interp1d as interp
from astropy.table import Table

In [2]:
v=20.
g=9.8
times=np.linspace(.1,10,10)
my_model= lambda v,g,t:v*t-.5*g*t**2
heights=v*times-.5*g*times**2
my_data=Table([times,heights,.1*heights],names=('time','height','error'))

In [3]:
def bounds_and_priors_to_ppf(bounds,priors=None):
    ppfs={}
    for key, val in six.iteritems(bounds):
        if key in ppfs:
            continue  # ppfs take priority over bounds/priors
        a, b = val
        if priors is not None and key in priors:
            # solve ppf at discrete points and return interpolating
            # function
            x_samples = np.linspace(0., 1., 101)
            ppf_samples = ppf(priors[key], x_samples, a, b)
            f = Interp1D(0., 1., ppf_samples)
        else:
            f = Interp1D(0., 1., np.array([a, b]))
        ppfs[key] = f
    return ppfs

def prior_transform(u):
        d = {}
        for i in range(npdim):
            d[iparam_names[i]] = ppflist[i](u[i])
        v = np.empty(ndim, dtype=np.float)
        for i in range(ndim):
            key = vparam_names[i]
            if key in d:
                v[i] = d[key]
            else:
                v[i] = tied[key](d)
        return v

def chisquare(observed_values,expected_values,errors):
    z = (observed_values - expected_values) / errors 
    chi2 = np.sum(z ** 2)
    return(chi2)

def loglike(parameters):
        return -0.5 * chisquare(my_data['height'],
                                my_model(parameters[0],parameters[1],my_data['time']),
                                my_data['error'])
    
def _get_marginal_pdfs( res, nbins=51, verbose=True ):
    """ Given the results <res> from a nested sampling chain, determine the
    marginalized posterior probability density functions for each of the
    parameters in the model.
    :param res:  the results of a nestlc run
    :param nbins: number of bins (steps along the x axis) for sampling
       each parameter's marginalized posterior probability
    :return: a dict with an entry for each parameter, giving a 2-tuple containing
       NDarrays of length nbins.  The first array in each pair gives the parameter
       value that defines the left edge of each bin along the parameter axis.
       The second array gives the posterior probability density integrated
       across that bin.
    """
    vparam_names = res.vparam_names
    weights = res.weights
    samples = res.samples

    pdfdict = {}

    for param in vparam_names :
        ipar = vparam_names.index( param )
        paramvals = samples[:,ipar]

        if nbins>1:
            if param in res.bounds :
                parvalmin, parvalmax = res.bounds[param]
            else :
                parvalmin, parvalmax = 0.99*paramvals.min(), 1.01*paramvals.max()
            parambins = np.linspace( parvalmin, parvalmax, nbins, endpoint=True ).flatten()
            binindices = np.digitize( paramvals, parambins )

            # we estimate the marginalized pdf by summing the weights of all points in the bin,
            # where the weight of each point is the prior volume at that point times the
            # likelihood, divided by the total evidence
            pdf = np.array( [ weights[np.where( binindices==ibin )].sum() for ibin in range(len(parambins)) ] )
        else :
            parambins = None
            pdf = None


        mean = (weights  * samples[:,ipar]).sum()
        #print(samples[:,ipar]-mean)
        #print(weights)
        std = np.sqrt( (weights * (samples[:,ipar]-mean)**2 ).sum() )


        pdfdict[param] = (parambins,pdf,mean,std,res.logz)

In [5]:
npoints=100
method='single'
maxiter=None
maxcall=None
rstate=None
verbose=False
vparam_names = ['v','g']
bounds={'v':(0,100),'g':(0,100)}
ppfs=bounds_and_priors_to_ppf(bounds)
iparam_names = [key for key in vparam_names if key in ppfs]
ppflist = [ppfs[key] for key in iparam_names]
npdim = len(iparam_names)  # length of u
ndim = len(vparam_names)  # length of v
res = nestle.sample(loglike, prior_transform, ndim, npdim=npdim,
                        npoints=npoints, method=method, maxiter=maxiter,
                        maxcall=maxcall, rstate=rstate,
                        callback=(nestle.print_progress if verbose else None))
vparameters, cov = nestle.mean_and_cov(res.samples, res.weights)
print(vparam_names,vparameters)
print(vparam_names,cov)

['v', 'g'] [19.97309592  9.78797504]
['v', 'g'] [[0.44430317 0.1991272 ]
 [0.1991272  0.09496342]]
